In [1]:
import numpy as np
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl
import cv2

# %matplotlib inline
%matplotlib qt

%config InlineBackend.figure_format = 'retina'

In [2]:
def load_image():    
    path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-1\A1data\data\compEx4.jpg'
    return cv2.imread(path)

In [3]:
def mat_to_np():    
    path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-1\A1data\data\compEx4.mat'
    mat_data = loadmat(path)
    corners = np.array(mat_data['corners'])
    K = np.array(mat_data['K'])
    v = np.array(mat_data['v'])
    return corners, K ,v

In [4]:
def to_cart(x):
    return x/x[-1]

In [5]:
def normalize(v):
    return v/(v @ v)**0.5

In [6]:
def compute_camera_center(P):
    M = P[:,:3]
    P4 = P[:,-1]
    C = -1*(np.linalg.inv(M) @ P4)
    return C

In [7]:
def compute_principal_axis(P):
    M = P[:,:3]
    m3 = P[-1,:3]
    v = np.linalg.det(M) * m3
    return normalize(v)

In [8]:
def camera_center_and_axis(P):
    C = compute_camera_center(P)
    p_axis = compute_principal_axis(P) 
    return C, p_axis

In [26]:
def calibrate(p, K):
    return np.linalg.inv(K) @ p

In [85]:
def compute_3D_point(pi, x2D):
    s = -np.transpose(pi[:-1]) @ x2D
    x3D = x2D / s
    return x3D

Before calibration: origin of the image is in the upper left corner.
After calibration: origin of the image is a bit upper left of the center of the image.

In [11]:
def plot_2D(corners, img, plt_img=False):
    x = corners[0,:]
    y = corners[1,:]

    plt.figure(figsize=(10,8))
    plt.plot(x, y, 'o', color='red', label='Corners')
    plt.axis('equal')

    if plt_img is True:
        plt.imshow(img)
    else:
        plt.gca().invert_yaxis()

    plt.legend(loc="lower right")
    plt.show()

In [129]:
def plot_3D(P1, P2, s1, s2, U1, U2):
    C1, p_axis1 = camera_center_and_axis(P1)
    C2, p_axis2 = camera_center_and_axis(P2)

    plt.figure(figsize=(10,8))
    ax = plt.axes(projection='3d')

    ax.plot(U1[0], U1[1], U1[2], 'o', color='magenta', label='Corners 3D')
    ax.plot(U2[0], U2[1], U2[2], 'o', color='green', label='Corners 2D')

    ax.plot(C1[0], C1[1], C1[2], 'o', color='red', label='Camera center 1')
    ax.plot(C2[0], C2[1], C2[2], 'o', color='lime', label='Camera center 2')

    x_p_axis1 = C1[0] + s1*p_axis1[0]
    y_p_axis1 = C1[1] + s1*p_axis1[1]
    z_p_axis1 = C1[2] + s1*p_axis1[2]

    x_p_axis2 = C2[0] + s2*p_axis2[0]
    y_p_axis2 = C2[1] + s2*p_axis2[1]
    z_p_axis2 = C2[2] + s2*p_axis2[2]

    ax.plot(x_p_axis1, y_p_axis1, z_p_axis1, 'o', color='blue', label='$s_1\\cdot$Principal axis 1')
    ax.plot(x_p_axis2, y_p_axis2, z_p_axis2, 'o', color='cyan', label='$s_2\\cdot$Principal axis 2')

    ax.plot([x_p_axis1,C1[0]], [y_p_axis1,C1[1]], [z_p_axis1,C1[2]], '-', color='black')
    ax.plot([x_p_axis2,C2[0]], [y_p_axis2,C2[1]], [z_p_axis2,C2[2]], '-', color='black')

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.axis('equal')
    # plt.gca().invert_yaxis()

    plt.legend(loc="lower right")
    plt.show()
    

In [89]:
def get_camera_and_scale_1():
    P1 = np.concatenate((np.identity(3), np.zeros([3,1])), 1)
    s1 = 3
    return P1, s1

In [134]:
def get_camera_and_scale_2():
    cos = np.cos(np.pi/6)
    sin = np.sin(np.pi/6)

    R = np.array([[cos,0,-sin],[0,1,0],[sin,0,cos]])
    C = np.array([-2,0,0])
    p4 = np.expand_dims((-C @ np.transpose(R)), 1)

    P2 = np.concatenate((R, p4), 1)
    s2 = 3.5
    return P2, s2

In [143]:
def compute_homography(P2, pi):
    R = P2[:,:3]
    t = P2[:,-1]
    return R - np.outer(t, np.transpose(pi[:-1]))

In [199]:
def plot_y_2D(y_H, y_P):
    
    plt.figure(figsize=(10,8))
    plt.plot(y_H[0,:], y_H[1,:], 'D', color='blue', label='$y=Hx$')
    plt.plot(y_P[0,:], y_P[1,:], '.', color='magenta', label='$y=P_2X$')

    plt.axis('equal')

    # if plt_img is True:
    #     plt.imshow(img)
    # else:
    plt.gca().invert_yaxis()

    plt.legend(loc="lower right")
    plt.show()

In [216]:
def transform_img(img, K, H):
    H_tot = K @ H @ np.linalg.inv(K)
    width = np.size(img, 1)
    height = np.size(img, 0)
    transf_img = cv2.warpPerspective(img, H_tot, (width, height))
    return transf_img

In [212]:
def plot_transf_img(img):

    plt.figure(figsize=(10,8))
    plt.axis('equal')

    # if plt_img is True:
    plt.imshow(img)
    # else:
    #     plt.gca().invert_yaxis()

    plt.show()

In [217]:
img = load_image()
corners, K, v = mat_to_np()
corners_cal = calibrate(corners, K)

# plot_2D(corners, img, plt_img=True)
# plot_2D(corners_cal, img)

corners_R3 = np.transpose(np.array([compute_3D_point(to_cart(v), corners_cal[:,i]) for i in range(np.size(corners_cal, 1))]))

P1, s1 = get_camera_and_scale_1()
P2, s2 = get_camera_and_scale_2()

# plot_3D(P1, P2, s1, s2, corners_3D, corners_cal)

corners_P3 = np.concatenate((corners_R3, np.ones([1,4])), 0)
H = compute_homography(P2, to_cart(v))
y_H = H @ corners_cal
y_P = P2 @ corners_P3

# plot_y_2D(to_cart(y_H), to_cart(y_P))

transf_img = transform_img(img, K, H)
plot_transf_img(transf_img)